# Análisis Exploratorio Simple

In [25]:
import pandas as pd
import numpy as np

## 1. Petróleo Brent

In [2]:
df_brent = pd.read_csv("../data/raw/brent_prices.csv")

In [3]:
df_brent.head()

,date,brent_price_usd
0,2022-01-03,78.980003
1,2022-01-04,80.000000
2,2022-01-05,80.800003
3,2022-01-06,81.989998
4,2022-01-07,81.750000


In [4]:
df_brent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 970 entries, 0 to 969
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             970 non-null    object 
 1   brent_price_usd  970 non-null    float64
dtypes: float64(1), object(1)
memory usage: 15.3+ KB


La variable "date" que corresponde a una fecha no trae el formato correcto.

In [5]:
# Convertir a datetime
df_brent['date'] = pd.to_datetime(df_brent['date'], format='%Y-%m-%d')

In [6]:
# Analizar si las fechas están completas

## 1. Rango de fechas
print("Fecha mínima:", df_brent['date'].min())
print("Fecha máxima:", df_brent['date'].max())
print("Total de registros:", len(df_brent))

## 2. Rango de fechas esperado vs real
fecha_min = df_brent['date'].min()
fecha_max = df_brent['date'].max()
rango_completo = pd.date_range(start=fecha_min, end=fecha_max, freq='B')

print("Fechas esperadas:", len(rango_completo))
print("Fechas reales:", df_brent['date'].nunique())


Fecha mínima: 2022-01-03 00:00:00
Fecha máxima: 2025-11-07 00:00:00
Total de registros: 970
Fechas esperadas: 1005
Fechas reales: 970


In [7]:
# 3. Identificar fechas faltantes
fechas_faltantes = rango_completo.difference(df_brent['date'])
print("Fechas faltantes:", len(fechas_faltantes))

if len(fechas_faltantes) > 0:
    print(f"\nPrimeras {min(10, len(fechas_faltantes))} fechas faltantes:")
    print(fechas_faltantes[:10])
    
    # Analizar si son festivos o días especiales
    print("\nDías de la semana de fechas faltantes:")
    print(pd.Series(fechas_faltantes).dt.day_name().value_counts())
else:
    print("\nNo hay fechas faltantes - Serie completa para días hábiles")


Fechas faltantes: 35

Primeras 10 fechas faltantes:
DatetimeIndex(['2022-01-17', '2022-02-21', '2022-04-15', '2022-05-30',
               '2022-06-20', '2022-07-04', '2022-09-05', '2022-11-24',
               '2022-12-26', '2023-01-02'],
              dtype='datetime64[ns]', freq=None)

Días de la semana de fechas faltantes:
Monday       23
Friday        4
Thursday      4
Wednesday     3
Tuesday       1
Name: count, dtype: int64


In [8]:
# 4. Verificar duplicados
duplicados = df_brent['date'].duplicated().sum()
print(f"Fechas duplicadas: {duplicados}")

Fechas duplicadas: 0


In [9]:
# 5. Verificar continuidad (diferencias entre fechas consecutivas)
df_sorted = df_brent.sort_values('date').reset_index(drop=True)
df_sorted['dias_diff'] = df_sorted['date'].diff().dt.days

print("Distribución de diferencias entre fechas consecutivas (en días):")
print(df_sorted['dias_diff'].value_counts().sort_index())

Distribución de diferencias entre fechas consecutivas (en días):
dias_diff
1.0    761
2.0      8
3.0    173
4.0     27
Name: count, dtype: int64


Las fechas faltantes suelen corresponder a festivos bursátiles (Navidad, Año Nuevo, etc.), lo cual es normal para datos financieros.

In [11]:
# Agregación mensual
df_brent["year"] = df_brent['date'].dt.year
df_brent["month"] = df_brent['date'].dt.month

df_monthly = df_brent.groupby(['year', 'month']).agg({
    'brent_price_usd' : [
        ('brent_price_mean', 'mean'),
        ('brent_price_min', 'min'),
        ('brent_price_max', 'max'),
        ('brent_price_median', 'median')
    ]
}).reset_index()

df_monthly.columns = ['year', 'month', 'brent_price_mean', 'brent_price_min', 'brent_price_max', 'brent_price_median']

# Redondear precios a 2 decimales
df_monthly[['brent_price_mean', 'brent_price_min', 'brent_price_max', 'brent_price_median']] = df_monthly[['brent_price_mean', 'brent_price_min', 'brent_price_max', 'brent_price_median']].round(2)
df_monthly.head()

,year,month,brent_price_mean,brent_price_min,brent_price_max,brent_price_median
0,2022,1,85.53,78.98,91.21,86.16
1,2022,2,94.03,89.16,100.99,93.28
2,2022,3,112.46,98.02,127.98,112.48
3,2022,4,105.92,98.48,113.16,106.65
4,2022,5,111.50,102.46,122.84,111.93


## 2. Datos de precios de la Secretaría de Energía de la Nación

Los archivos descargados de la Secretaría de Energía de la Nación contienen un registro detallado de precios de combustibles en estaciones de servicio de Argentina.

In [21]:
df_se = pd.read_csv("../data/raw/precios_eess_completo.csv")
df_se.head()

,Período,Operador,Nro Inscripción,Bandera,Fecha de baja,CUIT,Tipo Negocio,Dirección,Localidad,Provincia,...,NO Movimientos,Excentos,Impuesto Combustible Líquido,Impuesto Dióxido Carbono,Tasa Vial,tasa Municipal,Ingresos Brutos,Iva,Fondo fiduciario GNC,Impuesto Combustible Líquidos
0,2022/01,10 DE SETIEMBRE S.A.,1376,PUMA,NaN,33-64337382-9,Bocas de expendio (venta por menor) Duales (lí...,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,...,NO,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022/01,10 DE SETIEMBRE S.A.,1376,PUMA,NaN,33-64337382-9,Bocas de expendio (venta por menor) Duales (lí...,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,...,NO,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022/01,10 DE SETIEMBRE S.A.,1376,PUMA,NaN,33-64337382-9,Bocas de expendio (venta por menor) Duales (lí...,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,...,NO,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022/01,10 DE SETIEMBRE S.A.,1376,PUMA,NaN,33-64337382-9,Bocas de expendio (venta por menor) Duales (lí...,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,...,NO,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022/01,10 DE SETIEMBRE S.A.,1376,PUMA,NaN,33-64337382-9,Bocas de expendio (venta por menor) Duales (lí...,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,...,NO,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df_se["Producto"].value_counts()

Producto
Gas Oil Grado 2                     244741
Gas Oil Grado 3                     221282
Nafta (súper) entre 92 y 95 Ron     206350
Nafta (premium) de más de 95 Ron    192320
GNC                                  88763
N/D                                   8899
Kerosene                              7740
Nafta (común) hasta 92 Ron            2083
GLPA                                   155
Name: count, dtype: int64

De este dataframe nos interesa quedarnos con datos de precios de combustibles directo a consumidor final.

In [26]:
# Limpieza y filtrado inicial

## 1. Normalización de nombres de columnas
df_se.columns = df_se.columns.str.lower().str.replace(' ', '_').str.replace(".", "_")
df_se.columns = df_se.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

## 2. Corrección de tipo de dato
df_se["periodo"] = pd.to_datetime(df_se["periodo"], format="%Y/%m", errors='coerce')

## 3. Normalización nombre de productos
PRODUCTO_MAP = {
    "nafta (super) entre 92 y 95 ron": "NAFTA GRADO 2",
    "nafta (premium) de más de 95 ron": "NAFTA GRADO 3",
    "nafta (común) hasta 92 ron": "NAFTA GRADO 1",
    "gas oil grado 2": "GASOIL GRADO 2",
    "gas oil grado 3": "GASOIL GRADO 3",
    "gnc" : "GNC",
    "kerosene": "KEROSENE",
    "glpa": "GLPA",
    "n/d": np.nan
}

df_se["producto"] = df_se["producto"].str.lower().map(PRODUCTO_MAP)

In [27]:
print("Dimensiones del dataframe:", df_se.shape)

Dimensiones del dataframe: (972333, 26)


In [28]:
df_se.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 972333 entries, 0 to 972332
Data columns (total 26 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   periodo                        972333 non-null  datetime64[ns]
 1   operador                       972333 non-null  object        
 2   nro_inscripcion                972333 non-null  int64         
 3   bandera                        972333 non-null  object        
 4   fecha_de_baja                  34910 non-null   object        
 5   cuit                           972333 non-null  object        
 6   tipo_negocio                   972333 non-null  object        
 7   direccion                      972319 non-null  object        
 8   localidad                      972333 non-null  object        
 9   provincia                      972333 non-null  object        
 10  producto                       757084 non-null  object        
 11  

In [29]:
# Cantidad de duplicados
duplicados_se = df_se.duplicated().sum()
print(f"Cantidad de filas duplicadas: {duplicados_se}")

Cantidad de filas duplicadas: 20


In [30]:
# Seleccion de columnas relevantes
columnas_relevantes = ["periodo", "provincia", "bandera", "producto", "precio_surtidor", "volumen"]
df_se_filtered = df_se[columnas_relevantes]
df_se_filtered.head()

,periodo,provincia,bandera,producto,precio_surtidor,volumen
0,2022-01-01,BUENOS AIRES,PUMA,NaN,95.400002,6.740
1,2022-01-01,BUENOS AIRES,PUMA,NAFTA GRADO 3,109.900000,2.750
2,2022-01-01,BUENOS AIRES,PUMA,GASOIL GRADO 2,91.199997,9.930
3,2022-01-01,BUENOS AIRES,PUMA,GASOIL GRADO 3,106.400000,6.160
4,2022-01-01,BUENOS AIRES,PUMA,GNC,41.990002,36590.711


In [33]:
df_se_filtered = df_se_filtered.dropna(subset="producto")

In [34]:
# Agregaciones básicas por periodo, provincia, bandera y producto
agrupado = df_se_filtered.groupby(["periodo", "provincia", "bandera", "producto"]).agg(
    precio_surtidor_mediana=("precio_surtidor", "median"),
    volumen_total=("volumen", "sum")
)
agrupado = agrupado.reset_index()
agrupado.head()

,periodo,provincia,bandera,producto,precio_surtidor_mediana,volumen_total
0,2022-01-01,BUENOS AIRES,AGIRA,GASOIL GRADO 2,94.599998,95.206119
1,2022-01-01,BUENOS AIRES,AGIRA,GASOIL GRADO 3,110.800000,45.183570
2,2022-01-01,BUENOS AIRES,AGIRA,GNC,43.900002,409672.990000
3,2022-01-01,BUENOS AIRES,AGIRA,NAFTA GRADO 3,113.900000,74.430771
4,2022-01-01,BUENOS AIRES,ASPRO,GASOIL GRADO 2,94.049999,85.834002


In [32]:
agrupado["producto"].value_counts()

producto
GASOIL GRADO 2    7398
GASOIL GRADO 3    7398
NAFTA GRADO 3     7213
GNC               5185
KEROSENE          1826
NAFTA GRADO 1     1217
GLPA               153
Name: count, dtype: int64

In [35]:
agrupado.isna().sum()

periodo                    0
provincia                  0
bandera                    0
producto                   0
precio_surtidor_mediana    0
volumen_total              0
dtype: int64